In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import qp

In [ ]:
# pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
pz_dir = '/Users/echarles/pz'

In [ ]:
d = tables_io.read(f"{pz_dir}/data/on_sky/dp1_v29.0.0_gold_ECDFS.hdf5")

In [ ]:
flux_type = 'psf'
bands = 'ugrizy'

In [ ]:
def pretty_plot(d, flux_type, bands):
    fig = plt.figure()
    axes = fig.subplots(3, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        axs.hist(d[f"{band}_{flux_type}Mag"], np.linspace(15, 30, 151))
        axs.set_xlabel(f"{band} [mag]")
    fig.tight_layout()

In [ ]:
pretty_plot(d, flux_type, bands)

In [ ]:
t2 = tables_io.read(f'{pz_dir}/data/dp1_v29.0.0/5063/object.parquet')

In [ ]:
mask  = np.bitwise_and(
    np.bitwise_and(
        t2['i_extendedness'].to_numpy() == 1,
        t2['g_psfFlux'].to_numpy() / t2['g_psfFluxErr'].to_numpy() > 10,
    ),
    np.bitwise_and(
        t2['i_psfFlux'].to_numpy() / t2['i_psfFluxErr'].to_numpy() > 10,
        t2['r_psfFlux'].to_numpy() / t2['r_psfFluxErr'].to_numpy() > 10,
    ),
)
mask_extend = t2['i_extendedness'].to_numpy() == 1

In [ ]:
def fluxes_to_mags(
    fluxes: np.ndarray, zero_points: float | np.ndarray = 31.4
) -> np.ndarray:
    return -2.5 * np.log10(fluxes) + zero_points

def pretty_plot2(d, flux_type, bands, mask, mask_extend):
    fig = plt.figure()
    axes = fig.subplots(3, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        mags = fluxes_to_mags(d[f"{band}_{flux_type}Flux"])
        axs.hist(mags, np.linspace(15, 30, 151))
        axs.hist(mags[mask_extend], np.linspace(15, 30, 151))     
        axs.hist(mags[mask], np.linspace(15, 30, 151))
        
        axs.set_xlabel(f"{band} [mag]")
    fig.tight_layout()

In [ ]:
pretty_plot2(t2, 'psf', 'ugrizy', mask, mask_extend)

In [ ]:
def pretty_plot3(d, flux_type, bands, mask, mask_extend):
    fig = plt.figure()
    axes = fig.subplots(3, 2)
    for i, band in enumerate(bands):
        icol = i%2
        irow = int(i/2)
        axs = axes[irow][icol]
        sigmas = d[f"{band}_{flux_type}Flux"].to_numpy() / d[f"{band}_{flux_type}FluxErr"].to_numpy() 
        axs.hist(sigmas, np.logspace(-1, 3, 151))
        axs.hist(sigmas[mask_extend], np.logspace(-1, 3, 151))
        axs.hist(sigmas[mask], np.logspace(-1, 3, 151))
        axs.set_xscale('log')
        axs.set_xlabel(f"{band} [signif]")
    fig.tight_layout()

In [ ]:
pretty_plot3(t2, 'psf', 'ugrizy', mask, mask_extend)